# Exercise 6: Analysis of real data  (score 30 points)


In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
from imageio import imread
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import seaborn as sns
import os
from common import *
#from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.preprocessing import MinMaxScaler


print(f"Root directory: {os.getcwd()}")

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
    "font.size": 10,
})

%matplotlib inline 

Root directory: c:\Users\andre\Dropbox\FYS-STK4155_projects\FYS-STK4155 - Project1\FYS-STK4155-Prj1_report\code


# Global variables

In [2]:
np.random.seed(SEED_VALUE) # Random seed to guarantee reproducibility
# Paths
#INPUT_DATA = "data/input_data/"  # Path for input data
#REPORT_DATA = "data/report_data/" # Path for data ment for the report
#REPORT_FIGURES = "figures/" # Path for figures ment for the report
# Setting for range of degrees
from_degree = 0
to_degree = 14
degrees = np.arange(from_degree,to_degree)
degree = 5
# Setting for logspace range of lambdas 
from_lambda = -7 #
to_lambda = 3 #
nLambdas = 10
lambdas = np.logspace(from_lambda, to_lambda, nLambdas)
# Rescale settings
rescale = 0.1

# Reading and plotting terrain data

In [11]:
# Load the terrain
terrain1_file = "SRTM_data_Norway_1.tif"
terrain2_file = "SRTM_data_Norway_2.tif"
terrain1 = imread(f'{INPUT_DATA}{terrain1_file}')
terrain2 = imread(f'{INPUT_DATA}{terrain2_file}')

# Plotting terrain
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.title.set_text("Terrain over Norway 1")
ax1.set_xlabel("X"); ax1.set_ylabel("Y")
surf1 = ax1.imshow(terrain1, cmap='gray')
ax2.title.set_text("Terrain over Norway 2")
ax2.set_xlabel("X"); ax2.set_ylabel("Y")
surf2 = ax2.imshow(terrain2, cmap='gray')
plt.savefig(f"{REPORT_FIGURES}terrain_data.pdf")

In [15]:
#y = np.arange(0, 3500, 1)
#x = np.arange(0, 1500, 1)
# Make data.
z = np.array(terrain1)
scaler = MinMaxScaler()
scaler.fit(z)
z_scaled = scaler.transform(z)
print(np.max(z))
print(np.max(z_scaled))


#y = np.linspace(0,1,z.shape[0])
#x = np.linspace(0,1,z.shape[1])

y = np.arange(0, z.shape[0])
x = np.arange(0, z.shape[1])

x, y = np.meshgrid(x,y)


print(z.shape)



"""
fig = plt.figure()
# Ploting frankefunction without noise
ax1 = fig.add_subplot(1,2,1, projection='3d') # Are :)steike
ax1.title.set_text("Plot of the Franke Function")
ax1.view_init(elev=30., azim=-25.0)
ax1.set_xlabel("x"); ax1.set_ylabel("y"); ax1.set_zlabel("z")
surf1 = ax1.plot_surface(x,y,z, cmap=cm.coolwarm, linewidth = 0, antialiased=False)
# Customize the z axis.
ax1.set_zlim(-0.10, 1.40)
ax1.zaxis.set_major_locator(LinearLocator(10))
ax1.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
"""
%matplotlib
fig = plt.figure()
# Ploting frankefunction without noise
ax1 = fig.add_subplot(1,2,1, projection='3d') 
ax1.title.set_text("Terrain plot")
#ax1.view_init(elev=30., azim=-25.0)
ax1.set_xlabel("x"); ax1.set_ylabel("y"); ax1.set_zlabel("z")
ax1.plot_surface(x,y,z, cmap=cm.coolwarm, linewidth = 0, antialiased=False)


#fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Plot the surface.
#surf = ax.plot_surface(x, y, z_scaled, cmap=cm.coolwarm, linewidth=0, antialiased=False)

# Customize the z axis.
#ax.set_zlim(-1.01, 1.01)
#ax.zaxis.set_major_locator(LinearLocator(10))
# A StrMethodFormatter is used automatically
#ax.zaxis.set_major_formatter('{x:.02f}')

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)
plt.savefig(f"{REPORT_FIGURES}Terrain_plot.pdf")


1865
1.0000000000000002
(3601, 1801)
Using matplotlib backend: Qt5Agg


PermissionError: [Errno 13] Permission denied: '../figures/Terrain_plot.pdf'

In [5]:
fig = plt.figure()
# Ploting frankefunction without noise
ax1 = fig.add_subplot(1,2,1, projection='3d') # Are :)steike
ax1.title.set_text("Plot of the Franke Function")
ax1.view_init(elev=30., azim=-25.0)
ax1.set_xlabel("x"); ax1.set_ylabel("y"); ax1.set_zlabel("z")
surf1 = ax1.plot_surface(x,y,z, cmap=cm.coolwarm, linewidth = 0, antialiased=False)
# Customize the z axis.
ax1.set_zlim(-0.10, 1.40)
ax1.zaxis.set_major_locator(LinearLocator(10))
ax1.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# 1. Preprocessing and transformation
Least Square regression is not designed to tackle images directly. Thus, we must first transform the terrain data by slicing it into several bits and pieces. 

In [6]:
# Preprocess 



# Flattening the 2D images grid data
x_flat = x.ravel()
y_flat = y.ravel()


# 2. Constructing input data for least square regression
We construct the data for least square regression based on preprocessed data. We also set up variables that will be used throughout the exercise.

In [7]:
X = create_X(x_flat, y_flat, degree, intercept=False)
N,P = X.shape
nBetas = P
betas_OLS = np.empty((nLambdas, nBetas))
betas_Ridge = np.empty((nLambdas, nBetas))
betas_Lasso = np.empty((nLambdas, nBetas))
sigma_betas_OLS = np.empty((nLambdas, nBetas))
sigma_betas_Ridge = np.empty((nLambdas, nBetas))
sigma_betas_Lasso = np.empty((nLambdas, nBetas))

TypeError: create_X() got an unexpected keyword argument 'intercept'

In [ ]:
# trening
x_flat = x.ravel()
y_flat = y.ravel()
z_flat = z.ravel()
